In [ ]:
# Data Lake name
storageAccountName = "datalakeea7f393041fb9df4"

In [ ]:
display(dbutils.fs.ls(f"/mnt/{storageAccountName}/silver"))

path,name,size,modificationTime
dbfs:/mnt/datalakeea7f393041fb9df4/silver/application/,application/,0,0
dbfs:/mnt/datalakeea7f393041fb9df4/silver/bank_accounts/,bank_accounts/,0,0
dbfs:/mnt/datalakeea7f393041fb9df4/silver/cards/,cards/,0,0
dbfs:/mnt/datalakeea7f393041fb9df4/silver/transactions/,transactions/,0,0
dbfs:/mnt/datalakeea7f393041fb9df4/silver/users/,users/,0,0


In [ ]:
# Gerando DF baseado nos arquivos delta da camada silver
df_application = spark.read.format('delta').load(f"/mnt/{storageAccountName}/silver/application")
df_bank_accounts = spark.read.format('delta').load(f"/mnt/{storageAccountName}/silver/bank_accounts")
df_cards = spark.read.format('delta').load(f"/mnt/{storageAccountName}/silver/cards")
#df_loans = spark.read.format('delta').load(f"/mnt/{storageAccountName}/silver/loans")
df_transactions = spark.read.format('delta').load(f"/mnt/{storageAccountName}/silver/transactions")
df_users = spark.read.format('delta').load(f"/mnt/{storageAccountName}/silver/users")


In [ ]:
from pyspark.sql.functions import col

df_application.createOrReplaceTempView("df_application")

df_bank_accounts.createOrReplaceTempView("df_bank_accounts")

df_cards.createOrReplaceTempView("df_cards")

''' with spark.sql.builder.temporaryView("table2") as table2:
  df2.createOrReplaceTempView(table2) '''

df_transactions.createOrReplaceTempView("df_transactions")

df_users.createOrReplaceTempView("df_users")

join_cte = spark.sql("""
  SELECT t1.USERID, t1.APPLICATION_TYPE, t1.CREATED_AT as CREATED_AT_APPL, t1.INTEREST_RATE, t1.VALUE as VALUE_APPL,
  t2.BALANCE, t2.CREDIT_LIMIT,
  t3.CARD_CVV, t3.CARD_HOLDER, t3.CARD_MEMBER_SINCE, t3.CARD_NUMBER, t3.CARD_TYPE, t3.CARD_VALIDATION_DATE,
  t4.BALANCE_CHANGE_TYPE, t4.CREATED_AT as CREATED_AT_BA, t4.TYPE, t4.VAlUE as VALUE_BA,
  t5.PERSONID
  FROM df_application t1
   JOIN df_bank_accounts t2 ON t1.USERID = t2.USERID
   JOIN df_cards t3 on t1.USERID = t3.USERID
   JOIN df_transactions t4 on t1.USERID = t4.USERID
   JOIN df_users t5 on t1.USERID = t5.ID
""")

In [ ]:
join_cte.write.option(
    "path", f"dbfs:/mnt/{storageAccountName}/gold/obt"
)

In [ ]:
join_cte.write.mode("overwrite").saveAsTable("pipelinelinda.dashdb.obt")